### Passo 1 - Importar Arquivos e Bibliotecas

In [15]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import pathlib

In [ ]:
emails = pd.read_excel(r'/Users/administrador/Downloads/Projeto AutomacaoIndicadores/Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'/Users/administrador/Downloads/Projeto AutomacaoIndicadores/Bases de Dados/Lojas.csv', encoding='latin-1', sep=';')
vendas = pd.read_excel(r'/Users/administrador/Downloads/Projeto AutomacaoIndicadores/Bases de Dados/Vendas.xlsx')

vendas = vendas.merge(lojas, on='ID Loja')

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

dia = vendas['Data'].max()

### Passo 3 - Salvar a planilha na pasta de backup

In [18]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')
pastas_backup = caminho_backup.iterdir()

nomes_pastas = [arquivo.name for arquivo in pastas_backup]

for loja in dic_lojas:
    if loja not in nomes_pastas:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    nome_arquivo = f'{dia.month}_{dia.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dic_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [19]:
# Metas

meta_fat_dia = 1000
meta_fat_ano = 1650000

meta_qtd_prod_dia = 4
meta_qtd_prod_ano = 120

meta_ticket_dia = 500
meta_ticket_ano = 500

In [27]:
# Indicadores e email

for loja in dic_lojas:

    vendas_loja = dic_lojas[loja]
    vendas_dia = vendas_loja_ano.loc[vendas_loja_ano['Data']==dia, :]

    #faturamento ano e dia
    faturamento = vendas_loja['Valor Final'].sum()
    fatu_dia = vendas_dia['Valor Final'].sum()

    #divers de produtos ano e dia

    total_produtos_ano = len(vendas_loja_ano['Produto'].value_counts())
    total_produtos_dia = len(vendas_dia['Produto'].value_counts())

    #ticket medio ano e dia

    total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
    total_vendas_dia = vendas_dia.groupby('Código Venda').sum()

    ticket_medio_ano = total_vendas_ano['Valor Final'].mean()
    ticket_medio_dia = total_vendas_dia['Valor Final'].mean()

    # Configurações do e-mail
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    sender = 'tarikhadi7@gmail.com'
    receiver = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    subject = f'One Page Dia {dia.day}/{dia.month} - Loja {loja}'
    attachment_path = pathlib.Path.cwd() / caminho_backup / loja / f'{dia.month}_{dia.day}_{loja}.xlsx'
    
    if fatu_dia >= meta_fat_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento >= meta_fat_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if total_produtos_dia >= meta_qtd_prod_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if total_produtos_ano >= meta_qtd_prod_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticket_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticket_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
        
    body = f'''
        <p>Bom dia, {nome}</p>

        <p>O resultado de ontem <strong>({dia.day}/{dia.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

        <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
          </tr>
          <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${fatu_dia:.2f}</td>
            <td style="text-align: center">R${meta_fat_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
          </tr>
          <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{total_produtos_dia}</td>
            <td style="text-align: center">{meta_qtd_prod_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
            <td style="text-align: center">R${meta_ticket_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
          </tr>
        </table>
        <br>
        <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
          </tr>
          <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento:.2f}</td>
            <td style="text-align: center">R${meta_fat_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
          </tr>
          <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{total_produtos_ano}</td>
            <td style="text-align: center">{meta_qtd_prod_ano}</td>
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
            <td style="text-align: center">R${meta_ticket_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
          </tr>
        </table>

        <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

        <p>Qualquer dúvida estou à disposição.</p>
        <p>Att., Lira</p>
        '''

    # Criação do objeto de mensagem
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = receiver
    message['Subject'] = subject

    # Adiciona o corpo do e-mail
    message.attach(MIMEText(body, 'html'))

    # Leitura do arquivo a ser anexado
    attachment = open(attachment_path, 'rb')

    # Criação do objeto MIMEBase
    mimebase = MIMEBase('application', 'octet-stream')
    mimebase.set_payload(attachment.read())

    # Codifica o arquivo em base64
    encoders.encode_base64(mimebase)

    # Define o cabeçalho do anexo
    mimebase.add_header('Content-Disposition', f'attachment; filename="{attachment_path}"')

    # Adiciona o anexo à mensagem
    message.attach(mimebase)

    # Estabelece a conexão com o servidor SMTP do Gmail
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    username = 'tarikhadi7@gmail.com'
    password = 'fwpvmthkczyvemll'

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(username, password)

    # Envio do e-mail
    server.sendmail(sender, receiver, message.as_string())
    server.quit()
    print(f'Email da loja {loja} enviado.')

/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_dia = vendas_dia.groupby('Código Venda').sum()


Email da loja Iguatemi Esplanada enviado.


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_dia = vendas_dia.groupby('Código Venda').sum()


Email da loja Shopping Midway Mall enviado.


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_dia = vendas_dia.groupby('Código Venda').sum()


...

Email da loja Salvador Shopping enviado.


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_dia = vendas_dia.groupby('Código Venda').sum()


Email da loja Rio Mar Shopping Fortaleza enviado.


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_ano = vendas_loja.groupby('Código Venda').sum()
/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/330880005.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_vendas_dia = vendas_dia.groupby('Código Venda').sum()


SMTPServerDisconnected: Connection unexpectedly closed

### Passo 5 - Enviar por e-mail para o gerente

### Passo 7 - Criar ranking para diretoria

In [70]:
fat_lojas_ano_ranking = vendas.groupby('Loja')[['Loja','Valor Final']].sum().sort_values('Valor Final', ascending=False)
display(fat_lojas)

nome_arquivo = f'{dia.day}_{dia.month}_Ranking Anual.xlsx'
fat_lojas_ano_ranking.to_excel(f'Backup Arquivos Lojas/{nome_arquivo}')

fat_lojas_dia_ranking = vendas.loc[vendas['Data'] == dia, :].groupby('Loja')[['Loja','Valor Final']].sum().sort_values('Valor Final', ascending=False)
display(fat_lojas_dia)

nome_arquivo = f'{dia.day}_{dia.month}_Ranking Dia.xlsx'
fat_lojas_dia_ranking.to_excel(f'Backup Arquivos Lojas/{nome_arquivo}')

display(fat_lojas_ano_ranking.iloc[0, 0])


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/1069385959.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fat_lojas_ano_ranking = vendas.groupby('Loja')[['Loja','Valor Final']].sum().sort_values('Valor Final', ascending=False)


,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


/var/folders/ct/vm1dw8k165xfctx2hdsx3bc00000gn/T/ipykernel_733/1069385959.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fat_lojas_dia_ranking = vendas.loc[vendas['Data'] == dia, :].groupby('Loja')[['Loja','Valor Final']].sum().sort_values('Valor Final', ascending=False)


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


1762419

### Passo 8 - Enviar e-mail para diretoria

In [72]:

sender = 'tarikhadi7@gmail.com'
receiver = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
subject = f'Ranking Dia {dia.day}/{dia.month}'
body = f'''

Preazdos, Bom dia!

Melhor loja do ano em faturamento: Loja {fat_lojas_ano_ranking.index[0]} com Faturamento R$ {fat_lojas_ano_ranking.iloc[0, 0]:,.2f}
Pior loja do ano em faturamento: Loja {fat_lojas_ano_ranking.index[-1]} com Faturamento R$ {fat_lojas_ano_ranking.iloc[-1, 0]:,.2f}

Melhor loja do dia em faturamento: Loja {fat_lojas_dia_ranking.index[0]} com Faturamento R$ {fat_lojas_dia_ranking.iloc[0, 0]:,.2f}
Pior loja do dia em faturamento: Loja {fat_lojas_dia_ranking.index[-1]} com Faturametno R$ {fat_lojas_dia_ranking.iloc[-1, 0]:,.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas. 
Qualquer dúvida estou a disposição.

Att,
Tárik

''' 
attachment_paths = [
    pathlib.Path.cwd() / caminho_backup / f'{dia.day}_{dia.month}_Ranking Dia.xlsx', 
    pathlib.Path.cwd() / caminho_backup / f'{dia.day}_{dia.month}_Ranking Anual.xlsx'
]

# Criação do objeto de mensagem
message = MIMEMultipart()
message['From'] = sender
message['To'] = receiver
message['Subject'] = subject

# Adiciona o corpo do e-mail
message.attach(MIMEText(body, 'plain'))

for attachment_path in attachment_paths:

    with open(attachment_path, 'rb') as attachment:
        # Criação do objeto MIMEBase
        mimebase = MIMEBase('application', 'octet-stream')
        mimebase.set_payload(attachment.read())

        # Codifica o arquivo em base64
        encoders.encode_base64(mimebase)

        # Define o cabeçalho do anexo
        mimebase.add_header('Content-Disposition', f'attachment; filename="{attachment_path.name}"')

        # Adiciona o anexo à mensagem
        message.attach(mimebase)
        
# Estabelece a conexão com o servidor SMTP do Gmail
smtp_server = 'smtp.gmail.com'
smtp_port = 587
username = 'tarikhadi7@gmail.com'
password = 'fwpvmthkczyvemll'

server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(username, password)

# Envio do e-mail
server.sendmail(sender, receiver, message.as_string())
server.quit()
print(f'Email da Diretoria enviado.')

Email da Diretoria enviado.
